## 1. 导入依赖并设置随机种子

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import onnx

torch.manual_seed(42)

## 2. 定义实验模型

我们创建一个简单的模型，包含两个变量：
- `my_tensor`: 普通的 Tensor（即使设置了 `requires_grad=True`）
- `my_param`: nn.Parameter 包装的 Tensor

In [12]:
class ExperimentModel(nn.Module):
    def __init__(self):
        super().__init__()
        # 1. 定义一个普通的 Tensor
        # 即使我们手动设置 requires_grad=True，它也只是一个普通的张量
        # 为了演示效果，我们先设置为 False
        self.my_tensor = torch.randn(1, requires_grad=False)

        # 2. 定义一个 nn.Parameter
        # nn.Parameter 本质上也是 Tensor，但会自动注册到模型的 parameters() 中
        self.my_param = nn.Parameter(torch.randn(1))

        # 3. 注册一个缓冲区 (buffer)
        # 缓冲区不会被视为模型参数，但会随着模型一起保存和加载
        self.register_buffer('my_buffer', torch.randn(1))

        # 4. 注册一个持久化状态 (persistent state)
        # 这不是 Tensor，而是一个普通的 Python 对象
        self.register_buffer('my_state', torch.tensor(42), persistent=False)

        # 5. 注册一个参数，和 nn.Parameter 类似，但使用 register_parameter 方法
        self.register_parameter('my_registered_param', nn.Parameter(torch.randn(1)))

    def forward(self, x):
        # 简单的线性变换: y = x * tensor + param
        return x * self.my_tensor + self.my_param

model = ExperimentModel()
print("模型结构:")
print(model)

模型结构:
ExperimentModel()


## 3. 查看模型参数

关键观察：只有 `nn.Parameter` 会被注册到模型的参数列表中。

In [14]:
print("=== 模型眼中的参数 (model.named_parameters) ===")
for name, param in model.named_parameters():
    print(f"参数名: {name}")
    print(f"  - 值: {param.data}")
    print(f"  - 类型: {type(param)}")
    print(f"  - requires_grad: {param.requires_grad}")

=== 模型眼中的参数 (model.named_parameters) ===
参数名: my_param
  - 值: tensor([-1.6898])
  - 类型: <class 'torch.nn.parameter.Parameter'>
  - requires_grad: True
参数名: my_registered_param
  - 值: tensor([0.9580])
  - 类型: <class 'torch.nn.parameter.Parameter'>
  - requires_grad: True


## 4. 训练实验：观察更新行为

我们进行一次简单的训练步骤，观察哪些变量会被优化器更新。

In [16]:
# 创建优化器（只会优化 model.parameters() 中的变量）
optimizer = optim.SGD(model.parameters(), lr=1.0)

# 准备数据
data = torch.tensor([1.0])
target = torch.tensor([0.0])

# 记录初始值
init_tensor_val = model.my_tensor.item()
init_param_val = model.my_param.item()

print("=== 训练前的值 ===")
print(f"my_tensor: {init_tensor_val:.4f}")
print(f"my_param:  {init_param_val:.4f}")

=== 训练前的值 ===
my_tensor: 1.1103
my_param:  -1.6898


In [17]:
# 前向传播
output = model(data)
print(f"\n模型输出: {output.item():.4f}")

# 计算损失
loss = (output - target) ** 2
print(f"损失值: {loss.item():.4f}")

# 反向传播
loss.backward()

# 查看梯度
print("\n=== 梯度信息 ===")
print(f"my_tensor.grad: {model.my_tensor.grad}")
print(f"my_param.grad:  {model.my_param.grad}")


模型输出: -0.5795
损失值: 0.3358

=== 梯度信息 ===
my_tensor.grad: None
my_param.grad:  tensor([-1.1590])


In [18]:
# 优化器更新
optimizer.step()

print("\n=== 训练一步后的变化 ===")
print(f"my_tensor: {init_tensor_val:.4f} -> {model.my_tensor.item():.4f} (变化: {model.my_tensor.item() - init_tensor_val:.4f})")
print(f"my_param:  {init_param_val:.4f} -> {model.my_param.item():.4f} (变化: {model.my_param.item() - init_param_val:.4f})")

print("\n结论: 只有 nn.Parameter 被优化器更新了！")


=== 训练一步后的变化 ===
my_tensor: 1.1103 -> 1.1103 (变化: 0.0000)
my_param:  -1.6898 -> -0.5308 (变化: 1.1590)

结论: 只有 nn.Parameter 被优化器更新了！


## 5. 使用 requires_grad=True 的 Tensor

即使我们给普通 Tensor 设置 `requires_grad=True`，它仍然不会被优化器更新。

In [19]:
class ExperimentModel2(nn.Module):
    def __init__(self):
        super().__init__()
        # 这次设置 requires_grad=True
        self.my_tensor = torch.randn(1, requires_grad=True)
        self.my_param = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return x * self.my_tensor + self.my_param

model2 = ExperimentModel2()

print("=== requires_grad=True 的 Tensor ===")
print(f"my_tensor.requires_grad: {model2.my_tensor.requires_grad}")
print(f"my_param.requires_grad:  {model2.my_param.requires_grad}")

print("\n=== 模型参数列表 ===")
print(f"参数数量: {len(list(model2.parameters()))}")
for name, param in model2.named_parameters():
    print(f"  - {name}")

print("\n结论: 即使 requires_grad=True，普通 Tensor 也不会成为模型参数！")

=== requires_grad=True 的 Tensor ===
my_tensor.requires_grad: True
my_param.requires_grad:  True

=== 模型参数列表 ===
参数数量: 1
  - my_param

结论: 即使 requires_grad=True，普通 Tensor 也不会成为模型参数！


## 6. ONNX 导出分析

在导出为 ONNX 格式时，只有 `nn.Parameter` 会被保存为模型的初始化器（initializer）。

In [8]:
print("=== ONNX 导出分析 ===")
onnx_path = "/tmp/experiment_model.onnx"

# 导出模型
model.eval()
dummy_input = torch.tensor([1.0])

torch.onnx.export(
    model,
    (dummy_input,),
    onnx_path,
    input_names=["input_x"],
    output_names=["output"],
    opset_version=18,
)

print(f"模型已导出到: {onnx_path}")

=== ONNX 导出分析 ===


W1130 12:32:04.969000 59267 /Volumes/BitTopia/PyTorch/pytorch/torch/onnx/_internal/exporter/_registration.py:110] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `ExperimentModel()` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ExperimentModel()` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
模型已导出到: /tmp/experiment_model.onnx
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
模型已导出到: /tmp/experiment_model.onnx


/Users/howiewang/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/copyreg.py:99: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)


In [9]:
# 分析 ONNX 结构
model_onnx = onnx.load(onnx_path)
graph = model_onnx.graph

print("\n--- ONNX 图的 initializer (模型参数) ---")
if len(graph.initializer) == 0:
    print("  (无 initializer)")
else:
    for initializer in graph.initializer:
        arr = onnx.numpy_helper.to_array(initializer)
        print(f"\n名字: {initializer.name}")
        print(f"  - 值: {arr}")
        print(f"  - 形状: {initializer.dims}")
        print(f"  - 数据类型: {initializer.data_type}")


--- ONNX 图的 initializer (模型参数) ---

名字: my_param
  - 值: [-0.8021901]
  - 形状: [1]
  - 数据类型: 1

名字: my_tensor
  - 值: [0.33669037]
  - 形状: [1]
  - 数据类型: 1


In [10]:
# 分析 ONNX 节点
print("\n--- ONNX 图的节点 (计算图) ---")
for i, node in enumerate(graph.node):
    print(f"\n节点 {i}: {node.op_type}")
    print(f"  - 输入: {list(node.input)}")
    print(f"  - 输出: {list(node.output)}")


--- ONNX 图的节点 (计算图) ---

节点 0: Mul
  - 输入: ['input_x', 'my_tensor']
  - 输出: ['mul']

节点 1: Add
  - 输入: ['mul', 'my_param']
  - 输出: ['output']


### ONNX 导出观察

可以看到：
- `my_param` 被保存为 ONNX 的 initializer（模型的可学习参数）
- `my_tensor` 被当作常量直接嵌入到计算图中（或作为常量节点）

这意味着在部署时：
- Parameter 的值可以被轻松替换（如加载新的权重）
- Tensor 的值被硬编码到模型中，难以修改

## 7. 总结

| 特性 | 普通 Tensor | nn.Parameter |
|------|------------|-------------|
| **本质** | torch.Tensor | torch.Tensor（子类） |
| **requires_grad** | 需要手动设置 | 自动设置为 True |
| **注册到 model.parameters()** | ❌ 不会 | ✅ 自动注册 |
| **被优化器更新** | ❌ 不会 | ✅ 会被更新 |
| **保存/加载 state_dict** | ❌ 不会 | ✅ 会被保存 |
| **ONNX 导出** | 常量/硬编码 | initializer（参数） |
| **适用场景** | 固定值、中间变量 | 可学习的权重/偏置 |

### 关键要点

1. **nn.Parameter 是特殊的 Tensor**
   - 自动注册到模型的参数列表
   - 可被优化器自动发现和更新

2. **普通 Tensor 不是模型参数**
   - 即使设置 `requires_grad=True`，也不会被优化器更新
   - 适合存储常量或不需要训练的值

3. **在模型导出时的差异**
   - Parameter: 作为独立的权重被保存
   - Tensor: 作为常量嵌入计算图

4. **最佳实践**
   - 可学习的权重 → 使用 `nn.Parameter`
   - 固定常量 → 使用普通 `Tensor`
   - 中间计算结果 → 使用普通 `Tensor`

## 8. 实战示例：何时使用 nn.Parameter

让我们看一个实际的例子：自定义的带偏置的线性层。

In [11]:
class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        # 权重矩阵 - 需要训练
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        # 偏置向量 - 需要训练
        self.bias = nn.Parameter(torch.randn(out_features))
        # 缩放因子 - 固定值，不需要训练
        self.scale = torch.tensor(0.1)  # 普通 Tensor

    def forward(self, x):
        # y = (xW^T + b) * scale
        return (x @ self.weight.t() + self.bias) * self.scale

linear = CustomLinear(3, 2)

print("自定义线性层的参数:")
for name, param in linear.named_parameters():
    print(f"  - {name}: shape {param.shape}")

print("\n注意: scale 不在参数列表中（这是我们想要的效果）")

自定义线性层的参数:
  - weight: shape torch.Size([2, 3])
  - bias: shape torch.Size([2])

注意: scale 不在参数列表中（这是我们想要的效果）
